In [1]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import os
from glob import glob

Using TensorFlow backend.
/home/sandesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sandesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sandesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sandesh/anaconda3/lib/python3.7/site-packages/t

In [2]:
#directories
current_dir = os.getcwd()
train_dir = os.path.join(current_dir, 'Images/Train')
test_dir = os.path.join(current_dir, 'Images/Test')

In [3]:
batch_size = 16
img_rows, img_cols = 224,224
epochs = 10

In [4]:
#data augumentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.3,
                                  height_shift_range=0.3,
                                  shear_range=0.3,
                                  horizontal_flip=True,
                                  zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_rows, img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='sparse')
                                                    

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(img_rows, img_cols),
                                                  batch_size=batch_size,
                                                  class_mode='sparse')

Found 200 images belonging to 1 classes.
Found 41 images belonging to 1 classes.


In [5]:
folders = glob(current_dir + '/Images/Train/*')
len(folders)

1

In [6]:
#vgg16 transfer learning
vgg = VGG16(input_shape = [img_rows, img_cols, 3], weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
#to use weights of already trained layers
for layer in vgg.layers:
    layer.trainable = False

In [8]:
#flatten cnn layers 
ful_layer = Flatten()(vgg.output)

In [9]:
#fully connected dense layer
output = Dense(len(folders), activation='sigmoid')(ful_layer)

In [10]:
#create model
model = Model(inputs=vgg.input, outputs=output)

In [11]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [12]:
#compiling model
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [13]:
#train data
history = model.fit_generator(train_generator,
                              validation_data=test_generator,
                              epochs=epochs,
                              steps_per_epoch=20,
                              validation_steps=5
                             )

Epoch 1/10
20/20 [==============================] - 175s 9s/step - loss: 0.0000e+00 - accuracy: 0.6513 - val_loss: 0.0000e+00 - val_accuracy: 0.8082
Epoch 2/10
20/20 [==============================] - 181s 9s/step - loss: 0.0000e+00 - accuracy: 0.6378 - val_loss: 0.0000e+00 - val_accuracy: 0.8788
Epoch 3/10
20/20 [==============================] - 178s 9s/step - loss: 0.0000e+00 - accuracy: 0.6090 - val_loss: 0.0000e+00 - val_accuracy: 0.8030
Epoch 4/10
20/20 [==============================] - 179s 9s/step - loss: 0.0000e+00 - accuracy: 0.6184 - val_loss: 0.0000e+00 - val_accuracy: 0.8219
Epoch 5/10
20/20 [==============================] - 172s 9s/step - loss: 0.0000e+00 - accuracy: 0.6118 - val_loss: 0.0000e+00 - val_accuracy: 0.8485
Epoch 6/10
20/20 [==============================] - 174s 9s/step - loss: 0.0000e+00 - accuracy: 0.6118 - val_loss: 0.0000e+00 - val_accuracy: 0.8182
Epoch 7/10
20/20 [==============================] - 178s 9s/step - loss: 0.0000e+00 - accuracy: 0.6154 - v

In [15]:
model.save('face_recognition(VGG).h5')